# Formatting ETCSL TEI XML files
## Introduction

# TODO
- Revise all introductory text
- in using lxml consistently use .xpath() rather than .iter()
- try finding secondary and additional text with xpath() rather then in pre-processing
- note: 
`test = tree.xpath(".//w[preceding-sibling::addSpan[@type='secondary'] and following-sibling::anchor]/descendant-or-self::*[@form]")`

finds words (not lines). Adjust XML with

`for x in test:
    x.attrib["status"] = "secondary"`


The Electronic Text Corpus of Sumerian Literature ([ETCSL](http://etcsl.orinst.ox.ac.uk) 1998-2006) provides editions and translations of some 400 Sumerian literary texts. Goal of this Notebook is to format the ETCSL data in such a way that the (lemmatized) texts are made available for computational text analysis. In order to make the data compatible with output scraped from [ORACC](http://oracc.org), the Notebook ETCSL-to-EPSD2 should be run after running the current scraper.

For most purposes you do not need to run this scraper, because the final output is made available to you. However, if you need output in a different format or if you wish to know how the output was produced, you may read, adapt, and run this Notebook.

The original [ETCSL](http://etcsl.orinst.ox.ac.uk) files in TEI XML are available upon request from the [Oxford Text Archive](http://ota.ox.ac.uk/desc/2518). Note the following description on the OTA site:

> ## The Electronic Text Corpus of Sumerian Literature. Revised edition.

> ### Editor	
> Cunningham, Graham (ed.); Ebeling, Jarle (ed.); Black, Jeremy (deceased) (ed.); Flückiger-Hawker, Esther (ed.); Robson, Eleanor (ed.); Taylor, Jon (ed.); Zólyomi, Gábor (ed.)

> ### Availability	
> Use of this resource is restricted in some manner. Usually this means that it is available for non-commercial use only with prior permission of the depositor and on condition that this header is included in its entirety with any copy distributed.

The [manual](http://etcsl.orinst.ox.ac.uk/edition2/etcslmanual.php) of the ETCSL project explains in full detail the editorial principles and the technical details. According to the manual the ETCSL data are freely available and the XML source files can be downloaded.

The TEI XML source files were sent to me by the Oxford Text Archive upon request September 3rd 2015. Any (non-commercial) re-use of the data produced in this Notebook should reproduce the header quoted above ('Editor' and 'Availability') and is understood to be licensed under a [Creative Commons Share Alike](http://creativecommons.org/licenses/by-nc-sa/4.0/) license.



# The Scraper

This scraper expects the following files:

1. Directory Input
  * etcsl.txt  a list of ETCSL text numbers
2. Directory etcsl/transliterations/
  * This directory should contain the ETCSL TEI XML files.
3. Directory Equivalencies
  * ampersands.txt a list of HTML entities and their unicode equivalents
  * version_equivalencies.txt a list of ETCSL version names with their abbreviated forms.

The output is saved in the `Output` directory as a set of .txt files.

## 1. Setting Up
First import the proper packages: 

- re: Regular Expressions
- StringIO: enable treating strings as files (used for ElementTree)
- os: enable Python to perform basic Operating System functions (such as making a directory)
- ElementTree: read and analyze an XML file as an ordered tree
- time: allows the program to 'sleep' for a brief period
- tqdm: creates a progress bar

If you installed Python 3 and Jupyter by installing the [Anaconda Navigator](https://www.continuum.io/downloads), then most of these packages should already be installed, with the exception of tqdm. The first line in the cell below installs tqdm. It needs to be installed just once, after installing it you may invalidate that line by putting a # in front of it.

In [10]:
#! pip install tqdm
import re
from lxml import etree
import os
import time
import json
import pandas as pd
import tqdm
#import ipywidgets as widgets

#from ipywidgets import Checkbox, interactive
#from IPython.display import display
#from tqdm import *



## 1.a Load Equivalencies 

In [11]:
with open("equivalencies/equivalencies.json") as f:
    eq = json.load(f)
equiv = eq["suxwords"]
equiv.extend(eq["emesalwords"])
equiv.extend(eq["propernouns"])

## 2. Preprocessing 1: HTML-entities
The ETCSL TEI XML files are written in ASCII and represent special characters (such as š or ī) by a sequence of characters that begins with & and ends with ; (e.g. &c; represents š). These so-called HTML entities are used in translation, bibliography, and introductory text, but not in the transliteration of the Sumerian text itself (see below). The entities are for the most part project-specific and are declared and described elsewhere in the ETCSL file set. The ElementTree package cannot deal with these entities and thus we have to replace them with the actual (unicode) character that they represent, before feeding the data to ElementTree. 

All the entities are listed with their corresponding unicode character (or expression) in the file `Input/ampersands.txt` separated by a space:

    &aacute; á
    &aleph; ʾ
    &amacr; ā
    &ance; {anše}
    etc.
    
in the main process (below 11) the file `ampersands.txt` is read and made into the Python dictionary `findreplace` in which each of the HTML entities is a key, with its unicode equivalent as value. The function `ampersands()` uses this dictionary for a search-replace action.

The function `ampersands()` is called in `parsetext()` before the ElementTree is built. Note that the .xml files themselves are not changed by this process (or by any other process in this Notebook).

In [12]:
def ampersands(x):
    for amp in eq["ampersands"]:
        x = x.replace(amp, eq["ampersands"][amp])
    return x

## 4. Preprocessing 2: Marking 'Secondary Text' and/or 'Additional Text'

The ETCSL web pages include variants, indicated as '(1 ms. has instead: )', with the variant text enclosed in curly brackets. Two types of variants are distinguished: 'additional text' and 'secondary text'. 'Additional text' refers to a line that appears in a minority of sources (often in only one). 'Secondary text' refers to variant words or variant lines that are found in a minority of sources. The function `remove_extra()` may remove the words of 'secondary text' and/or 'additional text' before the text is parsed by ElementTree. 

In ETCSL TEI XML secondary/additional text is introduced by a tag of the type:
```xml
`<addSpan to="c141.v11" type="secondary"/>`
```
or
```xml
`<addSpan to="c141.v11" type="additional"/>`
```

The number c141 represents the text number in ETCSL (in this case Inana's Descent, text c.1.4.1). The return to the primary text is indicated by a tag of the type:
```xml
`<anchor id="c141.v11"/>`
```
Note that the `id` attribute in the `anchor` tag is identical to the `to` attribute in the `addSpan` tag.

The function `remove_extra()` uses regular expressions to identify and remove the Sumerian words and lines between those tags. The DOTALL flag (in re.DOTALL) allows the search in the regular expression to continue over multiple lines. Note that the function does not simply erase everything between the `<addSpan >` and the `<anchor >` tags identified with the regular expression. Instead, it indentifies words and lines between those tags, and elimibates those. This is the case because on occasion there may be tags that begin before the `<addSpan>` tag and end with the secondary region. Erasing the whole region would invalidate the XML structure and make it impossible to scan the data with ElementTree.

The function `remove_extra()` is called by the function `parse()` (see below, section 8). The third argument (`which`) is either `additional` (to remove additional text) or `secondary` (to remove secondary text). The default is not to remove anything.

In [13]:
def mark_extra(xmltext, textid, which):
    textid = textid.replace('.', '')
    find = re.compile('(<addSpan to=("' + textid + '.v[0-9]{1,3}") type="' + which + '"/>.*?<anchor id=\\2/>)', re.DOTALL)
    extra = re.findall(find, xmltext) # make a list of secondary/additional passages
    extra = [second[0] for second in extra] #findall creates a list of tuples; take the first of each tuple. The first
                                        # element in the tuple is the actual (secondary/additional) passage, 
                                        # the second is an id number.
    markwords = [re.sub('<w form=', '<w status="' + which + '" form=', instance) for instance in extra] 
                                        # mark status of each extra word in extra passage
    for search, repl in zip(extra, markwords):
        xmltext = xmltext.replace(search, repl)
    return xmltext

## 3. Transliteration Conventions

Transliteration of Sumerian text in ETCSL TEI XML files uses **c** for **š**, **j** for **ŋ** and regular numbers for index numbers. The function `tounicode()` replaces each of those. For example **cag4** is replaced by **šag₄**. This function is called in the function `getword()` to format citation forms and forms (transliteration). The function `tounicode` uses the dictionary `ascii_unicode` which is stored in the `equivalencies.json` file.

In [14]:
def tounicode(x):
    for char in eq["ascii_unicode"]:
        x = x.replace(char, eq["ascii_unicode"][char])
    return x

## Replace ETCSL by ORACC Lemmatization
For every word, once `cf` (Citation Form), `gw` (Guide Word), and `pos` (Part of Speech) have been pulled out of the XML file, it is run through the etcsl/oracc equivalence lists to match it with the oracc/epsd2 standards

In [15]:
def etcsl_to_oracc(word):
    lemma = {key:word[key] for key in ['cf', 'gw', 'pos']}
    for entry in equiv:
        if lemma == entry["etcsl"]:
            word['cf'] = entry["oracc"]["cf"]
            word["gw"] = entry["oracc"]["gw"]
            word["pos"] = entry["oracc"]["pos"]
            if "oracc2" in entry:
                word["cf2"] = entry["oracc2"]["cf"]
                word["gw2"] = entry["oracc2"]["gw"]
                word["pos2"] = entry["oracc2"]["pos"]
    return word

## 6. Formatting Words

A word in the ETCSL files is represented by a number of nodes in the XML tree that identify the form (transliteration), citation form, guide word, part of speech, etc. The function `getword()` formats the word as closely as possible to the ORACC conventions. Three different types of words are treated in three different ways: Proper Nouns, Sumerian words and Emesal words.

In ETCSL **proper nouns** are nouns, which are qualified by a 'type' (Divine Name, Personal Name, Geographical Name, etc.; abbreviated as DN, PN, GN, etc.). In ORACC a word has a single POS; for proper nouns this is DN, PN, GN, etc. - so what is 'type' in ETCSL becomes POS in ORACC. ORACC proper nouns usually do not have a guide word (only a number to enable disambiguation of namesakes). The ETCSL guide words ('label') for names come pretty close to ORACC citation forms. Names are therefore formatted differently from other nouns.

**Sumerian words** are treated in basically the same way in ETCSL and ORACC, but the citation forms and guide words are often different. Transformation of citation forms and guide words to ORACC (epsd2) standards takes place in the Notebook ETCSL-toEPSD2. This harmonization process uses a set of dictionaries (prepared by Niek Veldhuis and Terri Tanaka) that record ETCSL to EPSD2 equivalencies.

**Emesal words** in ETCSL use their Sumerian equivalents as citation form ('lemma'), adding a separate node ('emesal') for the Emesal form proper. This Emesal form is the one that is used as citation form in the output.

Guide words need removal of commas and spaces. Removal of commas will allow the output files to be read as Comma Separated Value (csv) files, which is an efficient input format for processes in Python and R. In the output file commas separate different fields from each other (text ID, text name, line number and text). Spaces need to be removed because standard tokenizers will understand spaces as word dividers. 

In [16]:
def getword(node, meta_d):
    word = {key:meta_d[key] for key in meta_d} # store all meta data in metad_d in the word dictionary
    if node.tag == 'gloss':
        form = "".join(node.itertext())
        form = form.replace('\n', ' ').strip()
        word["form"] = tounicode(form)
        word["lang"] = node.get("lang")
        return word
    
    word["cf"] = node.get('lemma').replace('Xbr', '(X)')
    word["gw"] = node.get('label')

    if node.get('pos'):
        word["pos"] = node.get('pos')
    else:
        word["pos"] = 'NA'
        word["gw"] = 'NA'
    form = node.get('form').replace('Xbr', '(X)')
    word["form"] = form
    
    if node.get('emesal'):
        word["cf"] = node.get('emesal')
        word["lang"] = "sux-x-emesal"
    else:
        word["lang"] = "sux"
    if word["pos"] != 'NU':
        word["cf"] = tounicode(word["cf"])
        word["form"] = tounicode(word["form"])
    if node.get('type') and word["pos"] == 'N':
        if node.get('type') != 'ideophone':
            word["pos"] = node.get('type')
            word["cf"] = node.get('label')
            word["gw"] = '1'
    if node.get('status'):
        word['status'] = node.get('status')

    word["gw"] = word["gw"].replace(",", ";") #remove commas from guide words (replace by semicolon) to prevent
                                            #problems with processing of the csv format
    word["gw"] = word["gw"].replace(" ", "-") #remove spaces from guide words (replace by hyphen). Spaces
                                            #create problems with tokenizers in computational text analysis.
    word = etcsl_to_oracc(word)   
    return word

## 7. Formatting Lines

Each line consist of a series of words. The function `getline()` iterates over a line, taking one word at a time. The words and their various features (language, citation form, guideword, part of speech and form) are retrieved calling the function `getword()`, which returns a dictionary. This dictionary is forwarded to the function `outputformat()` for formatting.

The function `getword()` will supply the Part of Speech 'X' to each word that has no POS tag already.

In [17]:
def getline(lnode, meta_d):
    meta_d["line_ref"] += 1
    if lnode.tag == 'gap':
        line = {key:meta_d[key] for key in ["id_text", "text_name", "version", "line_ref"]}
        line["extent"] = lnode.get("extent")
        line = [line]
        return line
    wordsinline = [] #initialize list for the words in this line
    for node in lnode.xpath('.//w|.//gloss[@lang="akk"]'):
                        # get <w> nodes and <gloss> nodes, but only Akkadian glosses
        word = getword(node, meta_d)
        if "cf2" in word:
            word2 = {key:word[key] for key in ["id_text", "text_name","version", "line_ref", "line_no",
                                               "form", "lang"]}
            word2["cf"] = word["cf2"]
            word2["gw"] = word["gw2"]
            word2["pos"] = word["pos2"]            
            word1 = {key:word[key] for key in ["id_text", "text_name","version", "line_ref", "line_no",
                                               "form", "lang", "cf", "gw", "pos"]}
            wordsinline.extend([word1, word2])
        else:
            wordsinline.append(word)
    return wordsinline

## 8. Sections

Some compositions are divided into **sections**. That is the case, in particular, when a composition has gaps of unknown length. 

The function `getsection()` is called by `getversion()` and receives three arguments: `tree` (an ElementTree object), `line_prefix` (which contains textid and the text name, and version name where applicable), and `csvformat` (which contains the header of the output CSV file). The function `getsection` checks to see whether a sub-division into sections is present. If so, it iterates over these sections. Each section (or, if there are no sections, the composition/version as a whole) consists of series of lines. The function `getline()` is called to request the contents of each line. The function returns the variable `csvformat`, which contains the formatted data.

In [18]:
def getsection(tree, meta_d):
    linesinsection = []
    sections = tree.find('.//div1')
    if sections != None: # if the text is not divided into sections - skip to else:
        for snode in tree.iter('div1'):
            section = snode.get('n')
            for lnode in snode.xpath('.//l|.//gap'):
                if lnode.tag == 'l':
                    line = section + lnode.get('n')
                    meta_d["line_no"] = line
                line = getline(lnode, meta_d)
                linesinsection.extend(line)
    else:
        for lnode in tree.xpath('.//l|.//gap'):
            if lnode.tag == 'l':
                line_no = lnode.get('n')
                meta_d["line_no"] = line_no
            line = getline(lnode, meta_d)
            linesinsection.extend(line)
    return linesinsection

## 9. Versions

In some cases an ETCSL file contains different versions of the same composition. The versions may be distinguished as 'Version A' vs. 'Version B' or may indicate the provenance of th version ('A version from Urim' vs. 'A version from Nibru'). In the edition of the proverbs the same mechanism is used to distinguish between numerous tablets (often lentils) that contain just one proverb, or a few, and are collected in the files "Proverbs from Susa," "Proverbs from Nibru," etc. (ETCSL c.6.2.1 - c.6.2.5).

The function `getversion()` is called by the function `parse()` and receives three arguments: `tree` (an ElementTree object), `line_prefix` (which contains the textid and the text name), and `csvformat` (which contains the header of the output CSV file). The function checks to see if versions are available in the file that is being parsed. If so, the function iterates over these versions while adding the version name to the variable `line_prefix`. If there are no versions, the version name is left empty. The parsing process is continued by calling `getsection()` to see if the composition/version is further divided into sections.

In [19]:
def getversion(tree, meta_d):
    sectionsinversion = []
    versions = tree.find('.//head')
    if versions != None: # if the text is not divided into versions - skip 'getversion()':
        for vnode in tree.iter('body'):
            version = vnode.find('head').text
            version = eq["versions"][version]
            meta_d["version"] = version
            section = getsection(vnode, meta_d)
            sectionsinversion.extend(section)
    else:
        meta_d["version"] = ''
        section = getsection(tree, meta_d)
        sectionsinversion.extend(section)
    return sectionsinversion


## 10. Parse a Text

The function `parsetext()` takes one xml file (a composition in ETCSL) and parses it, calling a variety of functions defined above. The function returns the variable `csvformat`. It contains a line-by-line representation of the text with version label (where applicable), line numbers (including section labels, where applicable) and all the lemmatized words.

The parsing is done by the ElementTree (ET) package. ET.parse expects a file, but instead it receives a variable here (`xmltext`). The function `StringIO()` allows a string to be treated as a file.

In [20]:
def parsetext(textid):
    meta_d = {"id_text": textid, "line_ref" : 0}
    with open('etcsl/transliterations/' + textid + '.xml') as f:
        xmltext = f.read()
    xmltext = ampersands(xmltext)          #replace HTML entities by Unicode equivalents
    xmltext = mark_extra(xmltext, textid, "additional")
    xmltext = mark_extra(xmltext, textid, "secondary")
    #xmltext = gaps(xmltext)
    #xmltext = glosses(xmltext)
    
    tree = etree.fromstring(xmltext)
    title = tree.xpath('.//title/descendant-or-self::text()')
    name = "".join(title) #some titles have children with <foreign> tag for Sumerian words
    name = name.replace(' -- a composite transliteration', '')
    name = name.replace(',', '')
    meta_d["text_name"] = name
    
    parsed = getversion(tree, meta_d)

    return parsed

## 11. Main Process

The code below opens a file `etcsl.txt` (in the directory `Input`) which contains all the numbers of ETCSL compositions (such as c.1.1.4). For each such number the corresponding xml file is opened and the content of the file is sent to the function `parsetext()`. `Parsetext()` returns the variabe `parsed` which is a list of dictionaries, each dictionary representing a single word in a single composition. The `parsed` is added to the list `alltexts`. In the end, `alltexts` will be a list of dictionaries that represent all the words in [ETCSL](http://etcsl.orinst.ox.ac.uk/). The list is transformed into a Pandas DataFrame. All missing values (`NaN`) are replaced by empty strings. The DataFrame is saved as a CSV file named `alltexts.csv` in the directory `output`.

In [12]:
with open("Input/etcsl.txt", "r") as f:
    textlist = f.read().splitlines()
if not os.path.exists('Output'):
    os.mkdir('Output')

alltexts = []
for eachtextid in tqdm.tqdm(textlist):
    parsed = parsetext(eachtextid)
    alltexts.extend(parsed)

df = pd.DataFrame(alltexts)
df = df.fillna('')
with open('output/alltexts.csv', 'w') as w:
    df.to_csv(w)

100%|██████████| 394/394 [01:40<00:00,  2.17it/s]
